In [ ]:

# Statistical Learning Project

In [ ]:
"""IMPORT OF PCK"""

import os
import glob
import tqdm
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)


import pandas as pd
import numpy as np
from scipy import stats
import math
import scipy.signal as signal
import random

from pathlib import Path
from IPython.core.display import display, HTML #display(HTML(data.to_html())) to show all variables

In [ ]:
"""DIRECTORY"""
os.getcwd()

## 1/ Import of data

In [ ]:
data_folder = Path(r"C:\Users\ville\PycharmProjects\stat_projetcs\Appr_Stat\data\Books_Dataset_20210108\archive")
bestsellers = data_folder / "bestsellers_with_categories.csv"
dta = pd.read_csv(bestsellers)

## 2/ Data Wrangling

In [ ]:
#Corrections for modelisation steps

dta["Genre"] = dta.Genre.replace(to_replace=['Fiction', 'Non Fiction'], value=[0, 1])
dta = dta.rename(columns={"User Rating": "Rating"})
dta.info()

In [ ]:
print(dta.head(5))
print(dta.tail(5))

<b>
<b>

We can see from the tail of our dataset that there are the same book, a multiple time.

The only difference is the year, in term of features. We can imagine that it represent the year when the book pop_up or the year when it becomes a bestsellers.
After a quick verification on internet, we can see that it represents: The year when the book was past of bestsellings.
Lets create a feature representing the time the book had been a bestsellings.

In the next steps, we will treat this problematic.

## -- Cleaning of Name column
Due to case sensitivity, the same book, with a different writing, can be misinterpreted to a different one.
We can correct this errors of case sensitivity by putting all the name in upper case.

In [ ]:
dta["Name"] = dta["Name"].str.upper()
#problem solved

## -- Cleaning of Author column

We identified some difference for the reference of some authors, such as J.K Rowling. To fix this error, we smooth out the author name to J.K Rowling (without space between ".")

In [ ]:
#To correct the space problem for few authors
for index, value in dta["Author"].items():
    if value.startswith("J."):
        change = value.replace(" ","")
        dta.loc[index, "Author"] = change
    else:
        pass


Creation of a new column which corresponds to the sum of year being best-sellings in total for the previous ten years.
Next we show the graph corresponding

In [ ]:
dta["sum_best"]=""
dta["sum_best"] = dta.groupby("Name").transform('count')

In [ ]:
plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta, x=dta.Name.value_counts(), palette="Set2");

for p in ax.patches:
    ax.annotate(format(p.get_height(), 'd'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Count");
ax.set_title("Times a Title Shows up on Top 50 Charts (2009-2019)");
plt.show();

In [ ]:
dta_repeat = pd.DataFrame([x, y] for x, y in dta.Author.value_counts().iteritems() if (y > 1))
dta_repeat.columns = ['Author','Top 50 Appearances']
dta_repeat.columns

In [ ]:
plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta_repeat, x="Top 50 Appearances", palette="Set2");

for p in ax.patches:
    ax.annotate('{:.1%}'.format(p.get_height()/len(dta_repeat)),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Count");
ax.set_title("Repeat Top 50 Appearances");
plt.show();


In [ ]:
#Looking for redundancy in the authors

plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta, x=dta.Author.value_counts(), palette="Set2")

for p in ax.patches:
    ax.annotate(format(p.get_height(), 'd'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Number of Authors")
ax.set_title("Total Appearances in the Top 50 Charts")
plt.show()

## -- Basic statistics

In [ ]:
dta.describe().transpose()

## -- Look for correlation

In [ ]:
dta.shape

numerical = list(set(dta.columns) -
                 set(['Name','Author']))
corr_matrix = dta[numerical].corr()

sns.heatmap(corr_matrix);
plt.xticks(rotation=45);

print(corr_matrix)

-- Add

In [ ]:
#Give an information on the dataset with the cosidered "best movies", with minimum 3000 review
data_order = dta.groupby(['Name', 'Author', 'Genre'], as_index=False)[['Rating', 'Reviews']].mean()
data_order = data_order[data_order['Reviews']>3000]
data_order = data_order.sort_values('Rating', ascending=False).head(20)
data_order


Use of weighted mean to attribute a review/rating which corresponds more to the reality field

with W_mean = ( ( v / m+v) * R) (( m / m+v)*C)

m = number of rating

C = mean Review

v= number of review

R = average numb of rating


In [ ]:
# Weighted rating
m = min(dta['Reviews'])
C = dta['Rating'].mean()
def weighted_rating(x, m=m, C=C):
    v = x['Reviews']
    R = x['Rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
dta['Weighted Rating'] = dta.apply(weighted_rating, axis=1)

In [ ]:
dta.groupby(['Name','Author','Genre'], as_index=False)[['Rating', 'Reviews', 'Weighted Rating']].mean().sort_values(by='Weighted Rating', ascending=False).head(10)

In [ ]:
dta.shape

numerical = list(set(dta.columns) -
                 set(['Name','Author']))
corr_matrix = dta[numerical].corr()

sns.heatmap(corr_matrix);
plt.xticks(rotation=45);

print(corr_matrix)
